In [1]:
from scipy.stats import norm
import numpy as np

# Black-Scholes定价公式
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# 隐含波动率的计算
def implied_volatility(option_price, S, K, T, r, option_type='call', tol=1e-6, max_iter=100):
    low, high = 1e-6, 5.0  # 波动率范围
    for _ in range(max_iter):
        mid = (low + high) / 2
        price = black_scholes(S, K, T, r, mid, option_type)
        if abs(price - option_price) < tol:
            return mid
        elif price < option_price:
            low = mid
        else:
            high = mid
    return mid

In [2]:
# 参数
S = 100  # 标的资产价格
K = 105  # 行权价
T = 100/360  # 到期时间
r = 0.05 # 无风险利率
option_price = 10  # 市场期权价格

# 计算隐含波动率
iv = implied_volatility(option_price, S, K, T, r, option_type='call')
print(f"隐含波动率: {iv:.2%}")

隐含波动率: 54.82%


In [3]:
from futu import *
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

2024-12-27 14:38:52,245 | 2976570 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=7465070


In [4]:
code='HK.00700'

In [5]:
ret, dates = quote_ctx.get_option_expiration_date(code=code)
dates

,strike_time,option_expiry_date_distance,expiration_cycle
0,2024-12-27,0,N/A
1,2024-12-30,3,N/A
2,2025-01-03,7,N/A
3,2025-01-27,31,N/A
4,2025-02-27,62,N/A
5,2025-03-28,91,N/A
6,2025-06-27,182,N/A
7,2025-09-29,276,N/A
8,2025-12-30,368,N/A


In [6]:
filter1 = OptionDataFilter()
filter1.delta_min = 0.3
filter1.delta_max = 0.7

In [7]:
date = dates.iloc[1]['strike_time']
date

'2024-12-30'

In [8]:
ret, options = quote_ctx.get_option_chain(code=code, start=date, end=date,option_type=OptionType.CALL, data_filter=filter1)

In [12]:
options[['code','name','strike_time','strike_price']]

,code,name,strike_time,strike_price
0,HK.TCH241230C420000,腾讯 241230 420.00 购,2024-12-30,420.0


In [24]:
option_code = options.iloc[0]['code']

In [25]:
ret, snaps = quote_ctx.get_market_snapshot([code, option_code])

In [26]:
snaps

,code,name,update_time,last_price,open_price,high_price,low_price,prev_close_price,volume,turnover,...,pre_turnover,pre_change_val,pre_change_rate,pre_amplitude,after_price,after_high_price,after_low_price,after_change_val,after_change_rate,after_amplitude
0,HK.00700,腾讯控股,2024-12-27 15:22:21,417.6,417.60,421.00,415.6,420.00,11704701,4.890758e+09,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,HK.TCH241230C420000,腾讯 241230 420.00 购,2024-12-27 15:21:14,1.4,1.46,3.02,1.0,3.08,4985,8.685380e+05,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [39]:
from datetime import datetime
days = (datetime.strptime(options.iloc[0]['strike_time'],'%Y-%m-%d') - datetime.now()).days
days

2

In [40]:
S = snaps.iloc[0]['last_price']
K = options.iloc[0]['strike_price']
T = days /220
r = 0.05
option_price = snaps.iloc[1]['last_price']

# 计算隐含波动率
iv = implied_volatility(option_price, S, K, T, r, option_type='call')
print(f"隐含波动率: {iv:.2%}")

隐含波动率: 14.70%


In [41]:
print('futu api iv', snaps.iloc[1]['option_implied_volatility'])

futu api iv 15.768


## 用对手盘计算 iv

In [27]:
ret_sub = quote_ctx.subscribe([code, option_code], [SubType.ORDER_BOOK], subscribe_push=False)[0]
if ret_sub != RET_OK:
    print('subscription failed')

In [42]:
ret, stock_data = quote_ctx.get_order_book(code, num=3)
stock_data

{'code': 'HK.00700',
 'name': '腾讯控股',
 'svr_recv_time_bid': '2024-12-27 15:34:55.545',
 'svr_recv_time_ask': '2024-12-27 15:34:55.545',
 'Bid': [(417.0, 15800, 54, {}),
  (416.8, 25000, 38, {}),
  (416.6, 24800, 41, {})],
 'Ask': [(417.2, 63800, 105, {}),
  (417.4, 28200, 29, {}),
  (417.6, 29800, 31, {})]}

In [43]:
ret, option_data = quote_ctx.get_order_book(option_code, num=3)
option_data

{'code': 'HK.TCH241230C420000',
 'name': '腾讯 241230 420.00 购',
 'svr_recv_time_bid': '2024-12-27 15:34:55.727',
 'svr_recv_time_ask': '2024-12-27 15:34:55.727',
 'Bid': [(1.21, 45, 1, {}), (1.2, 4, 2, {}), (1.19, 0, 0, {})],
 'Ask': [(1.38, 30, 1, {}), (1.39, 0, 0, {}), (1.4, 0, 0, {})]}

In [44]:
# ask/mid 中间价
mid_option_price = (option_data['Bid'][0][0] + option_data['Ask'][0][0]) /2
mid_stock_price = (stock_data['Bid'][0][0] + stock_data['Ask'][0][0]) /2
mid_option_price, mid_stock_price

(1.295, 417.1)

In [46]:
S = mid_stock_price
K = options.iloc[0]['strike_price']
T = days /220
r = 0.05
option_price = mid_option_price

# 计算隐含波动率
iv = implied_volatility(option_price, S, K, T, r, option_type='call')
print(f"隐含波动率: {iv:.2%}")

隐含波动率: 15.15%
